直接取回全部銀行指定匯率資訊

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.findrate.tw/USD/"
response = requests.get(url)
data = response.text

soup = BeautifulSoup(data, 'html.parser')
table = soup.find_all('table')[1]

# 提取訊息
exchange_rates = []
# 跳過標題行
for row in table.find_all('tr')[1:]:
    cols = row.find_all('td')
    # 檢查行是否有預期的列數
    if len(cols) == 7:
        exchange_rates.append([col.text.strip() for col in cols])

# 建立 DataFrame
if exchange_rates:
    df = pd.DataFrame(
        exchange_rates, 
        columns=[
            "銀行名稱", "現鈔買入", "現鈔賣出", "即期買入", 
            "即期賣出", "更新時間", "現鈔手續費"
        ]
    )
    # 儲存到 Excel
    df.to_excel("美金匯率.xlsx", index=False)
else:
    print("未找到資料或表格格式不同。")


優化以上代碼

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# 目標網址
url = "https://www.findrate.tw/USD/"

# 添加 User-Agent 避免被阻擋
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# 發送請求
response = requests.get(url, headers=headers)

# 確保請求成功
if response.status_code == 200:
    data = response.text
    soup = BeautifulSoup(data, 'html.parser')

    # 找到所有表格，避免索引錯誤
    tables = soup.find_all('table')
    
    if len(tables) > 1:
        # 目標表格
        table = tables[1]

        # 提取訊息
        exchange_rates = []
        # 跳過標題行
        for row in table.find_all('tr')[1:]:
            cols = row.find_all('td')
            # 確保符合預期格式
            if len(cols) == 7:
                exchange_rates.append([col.text.strip() for col in cols])

        # 建立 DataFrame
        if exchange_rates:
            df = pd.DataFrame(
                exchange_rates, 
                columns=[
                    "銀行名稱", "現鈔買入", "現鈔賣出", "即期買入", 
                    "即期賣出", "更新時間", "現鈔手續費"
                ]
            )

            # 生成 Excel 檔案名稱
            today_str = datetime.now().strftime('%Y%m%d')
            excel_filename = f"美金匯率_{today_str}.xlsx"

            # 儲存到 Excel
            df.to_excel(excel_filename, index=False)

            print(f"✅ 匯率資料已儲存至 {excel_filename}")
        else:
            print("⚠️ 未找到可用的匯率數據。")
    else:
        print("⚠️ 未找到目標表格，請確認網頁結構是否變更。")
else:
    print(f"❌ 無法獲取數據，HTTP 狀態碼: {response.status_code}")


✅ 匯率資料已儲存至 美金匯率_20250220.xlsx


加入日期

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def fetch_data():
    """從指定網站抓取美金匯率數據"""
    url = "https://www.findrate.tw/USD/"

    # 添加 `User-Agent` 以模擬瀏覽器，避免請求被阻擋
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    # 發送請求
    response = requests.get(url, headers=headers)

    # 檢查請求是否成功
    if response.status_code != 200:
        print(f"❌ 無法取得數據，HTTP 狀態碼: {response.status_code}")
        return []

    # 解析 HTML 內容
    soup = BeautifulSoup(response.text, 'html.parser')

    # 嘗試提取日期資訊
    date_info = soup.find('span', style="float:right")
    if not date_info:
        print("⚠️ 無法找到更新日期，請確認網頁結構是否變更。")
        return []

    # 解析日期
    date_text = date_info.text.strip().split('時間：')[-1]
    # 移除 `-` 轉為 YYYYMMDD 格式
    date_str = date_text.replace('-', '')

    # 找到所有表格，避免索引錯誤
    tables = soup.find_all('table')
    if len(tables) < 2:
        print("⚠️ 未找到目標表格，請確認網頁結構是否變更。")
        return []

    # 目標表格
    table = tables[1]

    # 提取匯率數據
    exchange_rates = []
    # 跳過標題行
    for row in table.find_all('tr')[1:]:
        cols = row.find_all('td')
        # 確保行數符合預期
        if len(cols) == 7:
            row_data = [col.text.strip() for col in cols]

            # 處理更新時間，移除 HTML 註解並合併日期
            update_time = BeautifulSoup(row_data[5], "html.parser").text
            # 完整時間資訊
            row_data[5] = f"{date_text} {update_time}"
            exchange_rates.append(row_data)
    # 回傳數據與日期字串
    return exchange_rates, date_str

# 抓取數據
exchange_rates, date_str = fetch_data()

if exchange_rates:
    # 建立 DataFrame
    df = pd.DataFrame(
        exchange_rates, 
        columns=[
            "銀行名稱", "現鈔買入", "現鈔賣出", "即期買入",
            "即期賣出", "更新時間", "現鈔手續費"
        ]
    )

    # 生成 Excel 檔案名稱
    excel_filename = f"美金匯率_V2_{date_str}.xlsx"

    # 儲存為 Excel 檔案
    df.to_excel(excel_filename, index=False)

    print(f"✅ 匯率資料已儲存至 {excel_filename}")
else:
    print("⚠️ 沒有找到可用的數據。")


✅ 匯率資料已儲存至 美金匯率_V2_20250220.xlsx


以下是從指定銀行取回指定資訊

In [5]:
from bs4 import BeautifulSoup
import requests

# 指定查詢的銀行網址（範例：台灣銀行）
url = "https://www.findrate.tw/bank/10/"

# 添加 `User-Agent` 以模擬瀏覽器，避免請求被阻擋
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# 發送請求
response = requests.get(url, headers=headers)

# 檢查請求是否成功
if response.status_code != 200:
    print(f"❌ 無法取得數據，HTTP 狀態碼: {response.status_code}")
else:
    # 解析 HTML 內容
    soup = BeautifulSoup(response.text, 'html.parser')

    # 找到包含匯率的表格
    table = soup.find('table', {'width': '725px'})
    
    # 確保表格存在
    if table:
        # 存放 USD 匯率資訊
        usd_info = None
        
        # 遍歷表格中的每一行，查找 USD
        for row in table.find_all('tr'):
            cols = row.find_all('td')
            if cols and 'USD' in cols[0].text:
                usd_info = [col.text.strip() for col in cols]
                break  # 找到後即停止搜尋
        
        # 顯示結果
        if usd_info:
            print("✅ 成功獲取美金匯率資訊：")
            print(usd_info)
        else:
            print("⚠️ 未找到 USD 匯率數據，請確認網頁結構是否變更。")
    else:
        print("⚠️ 未找到目標表格，請確認網頁結構是否變更。")


✅ 成功獲取美金匯率資訊：
['美金 USD', '32.365', '33.063', '32.68', '32.84']


在以上的查詢輸出中加入銀行資訊

In [6]:
from bs4 import BeautifulSoup
import requests

# 指定銀行查詢網址（範例：台灣銀行）
url = "https://www.findrate.tw/bank/10/"

# 添加 `User-Agent` 以模擬瀏覽器，避免請求被阻擋
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# 發送請求
response = requests.get(url, headers=headers)

# 檢查請求是否成功
if response.status_code != 200:
    print(f"❌ 無法取得數據，HTTP 狀態碼: {response.status_code}")
else:
    # 解析 HTML 內容
    soup = BeautifulSoup(response.text, 'html.parser')

    # 取得銀行名稱
    bank_title = soup.find('div', {'id': 'Title'})
    bank_name = bank_title.h1.text if bank_title else "未找到銀行名稱"

    # 取得跨行轉帳程式碼和 SWIFT 銀行國際程式碼
    bank_info_paragraph = bank_title.find_next_sibling('p') if bank_title else None
    if bank_info_paragraph:
        bank_transfer_code = bank_info_paragraph.find_all('b')[0].text if len(bank_info_paragraph.find_all('b')) > 0 else "未找到轉帳程式碼"
        swift_code = bank_info_paragraph.find_all('b')[1].text if len(bank_info_paragraph.find_all('b')) > 1 else "未找到 SWIFT 程式碼"
    else:
        bank_transfer_code = "未找到轉帳程式碼"
        swift_code = "未找到 SWIFT 程式碼"

    # 找到包含匯率的表格
    table = soup.find('table', {'width': '725px'})

    # 確保表格存在
    usd_info = None
    if table:
        # 遍歷表格中的每一行，查找 USD
        for row in table.find_all('tr'):
            cols = row.find_all('td')
            if cols and 'USD' in cols[0].text:
                usd_info = [col.text.strip() for col in cols]
                break  # 找到後即停止搜尋

    # 顯示結果
    print(f"🏦 銀行名稱: {bank_name}")
    print(f"🏧 跨行轉帳程式碼: {bank_transfer_code}")
    print(f"💳 SWIFT 程式碼: {swift_code}")
    
    if usd_info:
        print("✅ 美金匯率資訊:", usd_info)
    else:
        print("⚠️ 未找到 USD 匯率數據，請確認網頁結構是否變更。")


🏦 銀行名稱: 合作金庫牌告匯率表
🏧 跨行轉帳程式碼: 006
💳 SWIFT 程式碼: TACBTWTP
✅ 美金匯率資訊: ['美金 USD', '32.365', '33.063', '32.68', '32.84']


改查詢 `9`

In [13]:
from bs4 import BeautifulSoup
import requests

# 指定銀行查詢網址
url = "https://www.findrate.tw/bank/2/"

# 添加 `User-Agent` 以模擬瀏覽器，避免請求被阻擋
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# 發送請求
response = requests.get(url, headers=headers)

# 檢查請求是否成功
if response.status_code != 200:
    print(f"❌ 無法取得數據，HTTP 狀態碼: {response.status_code}")
else:
    # 解析 HTML 內容
    soup = BeautifulSoup(response.text, 'html.parser')

    # 取得銀行名稱
    bank_title = soup.find('div', {'id': 'Title'})
    bank_name = bank_title.h1.text if bank_title else "未找到銀行名稱"

    # 取得跨行轉帳程式碼和 SWIFT 銀行國際程式碼
    bank_info_paragraph = bank_title.find_next_sibling('p') if bank_title else None
    if bank_info_paragraph:
        bank_transfer_code = bank_info_paragraph.find_all('b')[0].text if len(bank_info_paragraph.find_all('b')) > 0 else "未找到轉帳程式碼"
        swift_code = bank_info_paragraph.find_all('b')[1].text if len(bank_info_paragraph.find_all('b')) > 1 else "未找到 SWIFT 程式碼"
    else:
        bank_transfer_code = "未找到轉帳程式碼"
        swift_code = "未找到 SWIFT 程式碼"

    # 找到包含匯率的表格
    table = soup.find('table', {'width': '725px'})

    # 確保表格存在
    usd_info = None
    if table:
        # 遍歷表格中的每一行，查找 USD
        for row in table.find_all('tr'):
            cols = row.find_all('td')
            if cols and 'USD' in cols[0].text:
                usd_info = [col.text.strip() for col in cols]
                break  # 找到後即停止搜尋

    # 顯示結果
    print(f"🏦 銀行名稱: {bank_name}")
    print(f"🏧 跨行轉帳程式碼: {bank_transfer_code}")
    print(f"💳 SWIFT 程式碼: {swift_code}")
    
    if usd_info:
        print("✅ 美金匯率資訊:", usd_info)
    else:
        print("⚠️ 未找到 USD 匯率數據，請確認網頁結構是否變更。")

🏦 銀行名稱: 中國信託牌告匯率表
🏧 跨行轉帳程式碼: 822
💳 SWIFT 程式碼: CTCBTWTP
⚠️ 未找到 USD 匯率數據，請確認網頁結構是否變更。


批次查詢多家銀行資訊

In [12]:
import requests
from bs4 import BeautifulSoup

def fetch_data(bank_index):
    """
    從指定的銀行索引頁面抓取銀行資訊與 USD 匯率數據
    """
    # 指定銀行查詢網址
    url = f"https://www.findrate.tw/bank/{bank_index}/"

    # 添加 `User-Agent` 以模擬瀏覽器，避免請求被阻擋
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    # 發送請求
    response = requests.get(url, headers=headers)

    # 檢查請求是否成功
    if response.status_code != 200:
        print(f"❌ 無法取得數據 (HTTP 狀態碼: {response.status_code})，跳過銀行索引 {bank_index}")
        return

    # 解析 HTML 內容
    soup = BeautifulSoup(response.text, 'html.parser')

    # 取得銀行名稱
    bank_title = soup.find('div', {'id': 'Title'})
    bank_name = bank_title.h1.text.strip() if bank_title else f"未找到銀行名稱 (索引: {bank_index})"

    # 取得跨行轉帳程式碼和 SWIFT 銀行國際程式碼
    bank_info_paragraph = bank_title.find_next_sibling('p') if bank_title else None
    if bank_info_paragraph:
        bank_transfer_code = bank_info_paragraph.find_all('b')[0].text if len(bank_info_paragraph.find_all('b')) > 0 else "未找到轉帳程式碼"
        swift_code = bank_info_paragraph.find_all('b')[1].text if len(bank_info_paragraph.find_all('b')) > 1 else "未找到 SWIFT 程式碼"
    else:
        bank_transfer_code = "未找到轉帳程式碼"
        swift_code = "未找到 SWIFT 程式碼"

    # 找到包含匯率的表格
    table = soup.find('table', {'width': '725px'})

    # 確保表格存在並提取 USD 匯率數據
    usd_info = None
    if table:
        for row in table.find_all('tr'):
            cols = row.find_all('td')
            if cols and 'USD' in cols[0].text:
                usd_info = [col.text.strip() for col in cols]
                break  # 找到後即停止搜尋

    # 顯示結果
    print("=" * 50)
    print(f"🏦 銀行名稱: {bank_name}")
    print(f"🏧 跨行轉帳程式碼: {bank_transfer_code}")
    print(f"💳 SWIFT 程式碼: {swift_code}")
    
    if usd_info:
        print("✅ 美金匯率資訊:", usd_info)
    else:
        print("⚠️ 未找到 USD 匯率數據，請確認網頁結構是否變更。")
    print("=" * 50)

# 主程式: 遍歷銀行索引 1~10
if __name__ == '__main__':
    for i in range(1, 11):  # 10 家銀行
        fetch_data(i)


🏦 銀行名稱: 上海商銀牌告匯率表
🏧 跨行轉帳程式碼: 011
💳 SWIFT 程式碼: SCSBTWTP
✅ 美金匯率資訊: ['美金 USD', '32.51', '33.01', '32.7', '32.81']
🏦 銀行名稱: 中國信託牌告匯率表
🏧 跨行轉帳程式碼: 822
💳 SWIFT 程式碼: CTCBTWTP
⚠️ 未找到 USD 匯率數據，請確認網頁結構是否變更。
🏦 銀行名稱: 京城銀行牌告匯率表
🏧 跨行轉帳程式碼: 054
💳 SWIFT 程式碼: TNBBTWTN
✅ 美金匯率資訊: ['美金 USD', '32.337', '33.027', '32.707', '32.807']
🏦 銀行名稱: 元大銀行牌告匯率表
🏧 跨行轉帳程式碼: 806
💳 SWIFT 程式碼: APBKTWTH
✅ 美金匯率資訊: ['美金 USD', '32.43', '33.044', '32.68', '32.816']
🏦 銀行名稱: 兆豐銀行牌告匯率表
🏧 跨行轉帳程式碼: 017
💳 SWIFT 程式碼: ICBCTWTP
✅ 美金匯率資訊: ['美金 USD', '32.35', '33.02', '32.69', '32.79']
🏦 銀行名稱: 匯豐銀行牌告匯率表
🏧 跨行轉帳程式碼: 081
💳 SWIFT 程式碼: HSBCTWTP
✅ 美金匯率資訊: ['美金 USD', '32.46', '33.06', '32.71', '32.81']
🏦 銀行名稱: 台中銀行牌告匯率表
🏧 跨行轉帳程式碼: 053
💳 SWIFT 程式碼: TCBBTWTH
✅ 美金匯率資訊: ['美金 USD', '32.459', '33.009', '32.709', '32.809']
🏦 銀行名稱: 富邦銀行牌告匯率表
🏧 跨行轉帳程式碼: 012
💳 SWIFT 程式碼: TPBKTWTP
✅ 美金匯率資訊: ['美金 USD', '32.425', '33.067', '32.687', '32.847']
🏦 銀行名稱: 台新銀行牌告匯率表
🏧 跨行轉帳程式碼: 812
💳 SWIFT 程式碼: TSIBTWTP
✅ 美金匯率資訊: ['美金 USD', '32.517', '32.994', '32.719', '32.819']
🏦 銀

存檔

In [14]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

def fetch_data(bank_index):
    """
    從指定的銀行索引頁面抓取銀行資訊與 USD 匯率數據
    """
    # 指定銀行查詢網址
    url = f"https://www.findrate.tw/bank/{bank_index}/"

    # 添加 `User-Agent` 以模擬瀏覽器，避免請求被阻擋
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    # 發送請求
    response = requests.get(url, headers=headers)

    # 檢查請求是否成功
    if response.status_code != 200:
        print(f"❌ 無法取得數據 (HTTP 狀態碼: {response.status_code})，跳過銀行索引 {bank_index}")
        return None

    # 解析 HTML 內容
    soup = BeautifulSoup(response.text, 'html.parser')

    # 取得完整銀行名稱
    bank_title = soup.find('div', {'id': 'Title'})
    full_bank_name = bank_title.h1.text.strip() if bank_title else f"未找到銀行名稱 (索引: {bank_index})"
    bank_name = full_bank_name.split('牌告匯率')[0].strip()  # 去除多餘字串

    # 取得跨行轉帳程式碼和 SWIFT 銀行國際程式碼
    bank_info_paragraph = bank_title.find_next_sibling('p') if bank_title else None
    if bank_info_paragraph:
        bank_transfer_code = bank_info_paragraph.find_all('b')[0].text if len(bank_info_paragraph.find_all('b')) > 0 else "未找到轉帳程式碼"
        swift_code = bank_info_paragraph.find_all('b')[1].text if len(bank_info_paragraph.find_all('b')) > 1 else "未找到 SWIFT 程式碼"
    else:
        bank_transfer_code = "未找到轉帳程式碼"
        swift_code = "未找到 SWIFT 程式碼"

    # 找到包含匯率的表格
    table = soup.find('table', {'width': '725px'})

    # 確保表格存在並提取 USD 匯率數據
    usd_info = None
    if table:
        for row in table.find_all('tr'):
            cols = row.find_all('td')
            if cols and 'USD' in cols[0].text:
                usd_info = [col.text.strip() for col in cols[1:]]  # 只提取匯率數據
                break  # 找到後即停止搜尋

    # 檢查是否有獲取到 USD 匯率數據
    if usd_info is not None:
        return [bank_name, bank_transfer_code, swift_code] + usd_info
    else:
        return [bank_name, bank_transfer_code, swift_code] + [None, None, None, None]  # 無數據則填入 None

# 初始化儲存所有銀行數據的列表
all_bank_data = []

# 遍歷 1~10 的銀行索引
for i in range(1, 11):
    result = fetch_data(i)
    if result:
        all_bank_data.append(result)

# 建立 DataFrame
df = pd.DataFrame(all_bank_data, columns=[
    "銀行名稱", "銀行程式碼", "SWIFT Code", "現金買入", "現金賣出", "即期買入", "即期賣出"
])

# 取得當前日期，格式為 YYYYMMDD
current_date = datetime.now().strftime("%Y%m%d")

# 生成 Excel 檔案名稱
file_name = f"美金匯率_彙整_{current_date}.xlsx"

# 儲存為 Excel 檔案
df.to_excel(file_name, index=False)

print(f"✅ 匯率數據已儲存至 {file_name}")


✅ 匯率數據已儲存至 美金匯率_彙整_20250220.xlsx


進行比較

In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def fetch_data(bank_index):
    """
    從指定的銀行索引頁面抓取銀行資訊與 USD 匯率數據
    """
    url = f"https://www.findrate.tw/bank/{bank_index}/"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"❌ 無法取得數據 (HTTP 狀態碼: {response.status_code})，跳過銀行索引 {bank_index}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    bank_title = soup.find('div', {'id': 'Title'})
    full_bank_name = bank_title.h1.text.strip() if bank_title else f"未找到銀行名稱 (索引: {bank_index})"
    bank_name = full_bank_name.split('牌告匯率')[0].strip()

    table = soup.find('table', {'width': '725px'})

    usd_info = None
    if table:
        for row in table.find_all('tr'):
            cols = row.find_all('td')
            if cols and 'USD' in cols[0].text:
                usd_info = [col.text.strip() for col in cols[1:]]  # 只提取匯率數據
                break  

    if usd_info:
        try:
            return [bank_name, float(usd_info[2]), float(usd_info[3])]  # 即期買入、即期賣出
        except ValueError:
            print(f"⚠️ {bank_name} 資料格式錯誤，跳過")
            return None
    else:
        return None

# 抓取所有銀行數據
all_bank_data = []
for i in range(1, 11):
    result = fetch_data(i)
    if result:
        all_bank_data.append(result)

# 建立 DataFrame
df = pd.DataFrame(all_bank_data, columns=["銀行名稱", "即期買入", "即期賣出"])

# 找出即期買入最高與最低
if not df.empty:
    max_buy = df.loc[df["即期買入"].idxmax()]
    min_buy = df.loc[df["即期買入"].idxmin()]
    max_sell = df.loc[df["即期賣出"].idxmax()]
    min_sell = df.loc[df["即期賣出"].idxmin()]

    print("\n📊 即期買入與即期賣出最高與最低銀行")
    print(f"🏆 即期買入最高: {max_buy['銀行名稱']} - {max_buy['即期買入']}")
    print(f"🔻 即期買入最低: {min_buy['銀行名稱']} - {min_buy['即期買入']}")
    print(f"🏆 即期賣出最高: {max_sell['銀行名稱']} - {max_sell['即期賣出']}")
    print(f"🔻 即期賣出最低: {min_sell['銀行名稱']} - {min_sell['即期賣出']}")
else:
    print("❌ 無法獲取有效的匯率數據")



📊 即期買入與即期賣出最高與最低銀行
🏆 即期買入最高: 台新銀行 - 32.719
🔻 即期買入最低: 元大銀行 - 32.68
🏆 即期賣出最高: 富邦銀行 - 32.847
🔻 即期賣出最低: 兆豐銀行 - 32.79
